# Introduction to `fastplotlib`

## `fastplotlib` API

### 1. Graphics - objects that are drawn
- `Image`, `Line`, `Scatter`, `Heatmap`
- Collections - `LineCollection`, `LineStack` (ex: neural timeseries data)
- Interactions

### 2. Layouts
- `Plot` - a single plot area 
- `GridPlot` - a grid of `Subplots`

### 3. Widgets - high level widgets to make repetitive UIs easier
- `ImageWidget`- n-dimensional widget for `Image` data
    - Sliders, support window functions, `GridPlot`, etc.
    
    

This notebook goes through basic components of the `fastplotlib` API including images, image updates, and `ImageWidget`.

**The example images are from imageio so you will need to install it for this example notebook. But imageio is not required to use fasptlotlib**

In [2]:
!pip install imageio

In [3]:
import imageio.v3 as iio

In [4]:
import fastplotlib as fpl
from ipywidgets import VBox, HBox, IntSlider
import numpy as np
from sidecar import Sidecar

### Simple image

In [5]:
# create a `Plot` instance
plot = fpl.Plot()

# get a grayscale image
data = iio.imread("imageio:camera.png")

# plot the image data
image_graphic = plot.add_image(data=data, name="sample-image")

# display plot in side car
sc = Sidecar(title="sample image")

with sc:
    # show the plot
    display(plot.show())

RFBOutputContext()

**Use the handle on the bottom right corner of the _canvas_ to resize it. You can also pan and zoom using your mouse!**

By default the origin is on the bottom left, you can click the flip button to flip the y-axis, or use `plot.camera.world.scale_y *= -1`

Changing graphic **"features"**

In [6]:
image_graphic.cmap = "viridis"

### Slicing data

**Most features, such as `data`, support slicing!**

Our image data is of shape [n_rows, n_columns]

In [7]:
image_graphic.data[::15, :] = 1
image_graphic.data[:, ::15] = 1

**Fancy indexing**

In [8]:
image_graphic.data[data > 175] = 255

**Adjust vmin vmax**

In [9]:
image_graphic.cmap.vmin = 50
image_graphic.cmap.vmax = 150

**Set the entire data array again**

Note: The shape of the new data array must match the current data shown in the `Graphic`

In [10]:
new_data = iio.imread("imageio:astronaut.png")
new_data.shape

(512, 512, 3)

This is an RGB image, convert to grayscale to maintain the shape of (512, 512)

In [11]:
gray = new_data.dot([0.3, 0.6, 0.1])
gray.shape

(512, 512)

In [12]:
image_graphic.data = gray

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


**reset vmin vmax**

In [13]:
image_graphic.cmap.reset_vmin_vmax()

### Indexing plots

**Plots are indexable and give you their graphics by name**

In [14]:
plot

unnamed: Plot @ 0x7f5a200b5dd0
  parent: None
  Graphics:
	'sample-image': ImageGraphic @ 0x7f59fcf64f90

In [15]:
plot["sample-image"]

<weakproxy at 0x7f59fcf5a750 to ImageGraphic at 0x7f59fcf64f90>

**You can also use numerical indexing on `plot.graphics`**

In [16]:
plot.graphics

(<weakproxy at 0x7f59fcf5a750 to ImageGraphic at 0x7f59fcf64f90>,)

In [17]:
plot.graphics[0]

<weakproxy at 0x7f59fcf5a750 to ImageGraphic at 0x7f59fcf64f90>

The `Graphic` instance is also returned when you call `plot.add_<graphic_type>`

In [18]:
image_graphic 

<weakproxy at 0x7f59fcf5a750 to ImageGraphic at 0x7f59fcf64f90>

In [20]:
image_graphic == plot["sample-image"]

True

In [21]:
# close sidecar
sc.close()

### RGB images are also supported

`cmap` arguments are ignored for rgb images, but vmin vmax still works

In [24]:
plot_rgb = fpl.Plot()

plot_rgb.add_image(new_data, name="rgb-image")

sc = Sidecar(title="rgb image")

with sc:
    display(plot_rgb.show())

RFBOutputContext()

In [25]:
plot_rgb["rgb-image"].cmap.vmin = 100

In [26]:
sc.close()

### Image updates

This examples show how you can define animation functions that run on every render cycle.

In [42]:
# create another `Plot` instance
plot_v = fpl.Plot()

plot.canvas.max_buffered_frames = 1

# make some random data again
data = np.random.rand(512, 512)

# plot the data
plot_v.add_image(data=data, name="random-image")

# a function to update the image_graphic
# a plot will pass its plot instance to the animation function as an arugment
def update_data(plot_instance):
    new_data = np.random.rand(512, 512)
    plot_instance["random-image"].data = new_data

#add this as an animation function
plot_v.add_animations(update_data)

# sidecar to display plot
sc = Sidecar(title="image updates")

with sc:
    # show the plot
    display(plot_v.show())

RFBOutputContext()

### We can share controllers across plots

This example creates a new plot, but it synchronizes the pan-zoom controller

In [43]:
plot_sync = fpl.Plot(controller=plot_v.controller)

data = np.random.rand(512, 512)

image_graphic_instance = plot_sync.add_image(data=data, cmap="viridis")

# you will need to define a new animation function for this graphic
def update_data_2():
    new_data = np.random.rand(512, 512)
    # alternatively, you can use the stored reference to the graphic as well instead of indexing the Plot
    image_graphic_instance.data = new_data

plot_sync.add_animations(update_data_2)

plot_sync.show()

RFBOutputContext()

#### Keeping a reference to the Graphic instance, as shown above `image_graphic_instance`, is useful if you're creating something where you need flexibility in the naming of the graphics

In [44]:
sc.close()

### You can also use `ipywidgets.VBox` and `HBox` to stack plots.

In [45]:
VBox([plot_v.show(), plot_sync.show()])

In [46]:
HBox([plot_v.show(), plot_sync.show()])

In [47]:
# close plots
plot_v.close()
plot_sync.close()

# Sliders to scroll through image data

We often already have large image arrays (whether in RAM or through lazy loading), and want to view 2D frames across one or more dimensions. There is an `ImageWidget` that should really be used for this, but this example just shows how you can use `ipywidgets` to change data or any **`GraphicFeature`**

## Plot and scroll through the first dimension with a slider

In [33]:
# use imagio cockatoo video
movie = iio.imread('imageio:cockatoo.mp4', index=None)

In [34]:
plot_movie = fpl.Plot()

# plot the first frame to initialize
movie_graphic = plot_movie.add_image(movie[0], vmin=0, vmax=movie.max(), cmap="gray")

# make a slider
slider = IntSlider(min=0, max=movie.shape[0] - 1, step=1, value=0)

# function to update movie_graphic
def update_movie(change):    
    index = change["new"]
    movie_graphic.data = movie[index]
    
slider.observe(update_movie, "value")
    
# Use an ipywidgets VBox to show the plot and slider
VBox([plot_movie.show(), slider])

RFBOutputContext()

#### Note that the use of globals in the `update_movie()` here can get messy, this is not recommended and  you should create a class to properly handle combining widgets like this. _However_ if you want slider widgets for imaging data the recommended way to do this is by using the `ImageWidget`.

## Single image sequence `ImageWidget` using zebrafish data

In [ ]:
# load in zebrafish data

In [13]:
iw = fpl.ImageWidget(
    data=a,
    vmin_vmax_sliders=True,
    cmap="viridis"
)

RFBOutputContext()

In [14]:
sc = Sidecar()

with sc:
    display(iw.show())

### can dynamically change features

In [15]:
iw.gridplot[0, 0].graphics[0].cmap = "gnuplot2"

### Play with setting different window functions

These can also be given as kwargs to `ImageWidget` during instantiation

In [16]:
# must be in the form of {dim: (func, window_size)}
iw.window_funcs = {"t": (np.mean, 13)}

In [17]:
# change the winow size
iw.window_funcs["t"].window_size = 23

In [18]:
# change the function
iw.window_funcs["t"].func = np.max

In [19]:
# or set it again
iw.window_funcs = {"t": (np.min, 11)}